In [1]:
pip show openai

Name: openai
Version: 0.28.1
Summary: Python client library for the OpenAI API
Home-page: https://github.com/openai/openai-python
Author: OpenAI
Author-email: support@openai.com
License: 
Location: c:\users\grzegorz.kosek\appdata\local\miniconda3\envs\gen_ai\lib\site-packages
Requires: aiohttp, requests, tqdm
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [2]:
def greet_user():
    # Ask the user for their name
    user_name = input("Please enter your name: ")
    
    # Print a greeting message
    print(f"Hello, {user_name}!")

# Call the function to start the process
greet_user()


Hello, Grzegorz!


In [ ]:
# Original function

import azure.functions as func
import logging

app = func.FunctionApp(http_auth_level=func.AuthLevel.ANONYMOUS)

@app.route(route="gen_ai")
def gen_ai(req: func.HttpRequest) -> func.HttpResponse:
    logging.info('Python HTTP trigger function processed a request.')

    name = req.params.get('name')
    if not name:
        try:
            req_body = req.get_json()
        except ValueError:
            pass
        else:
            name = req_body.get('name')

    if name:
        return func.HttpResponse(f"Hello, {name}! Poland GenAI Club is here!")
    else:
        return func.HttpResponse(
             "This HTTP triggered function executed successfully. Pass a name in the query string or in the request body for a personalized response.",
             status_code=200
        )

@app.route(route="hello", methods=["GET"])
def hello(req: func.HttpRequest) -> func.HttpResponse:
    logging.info('Python HTTP trigger function processed a request for /api/hello.')

    return func.HttpResponse("Hello, World!", status_code=200)

In [ ]:
import azure.functions as func
import logging
import openai
import os
from dotenv import load_dotenv

app = func.FunctionApp(http_auth_level=func.AuthLevel.ANONYMOUS)

# config
load_dotenv()
openai.api_type = "azure"
openai.api_base = "https://gen-ai-training-avanade-2.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")

@app.route(route="gen_ai")
def gen_ai(req: func.HttpRequest) -> func.HttpResponse:
    logging.info('Python HTTP trigger function processed a request.')

    name = req.params.get('name')
    if not name:
        try:
            req_body = req.get_json()
        except ValueError:
            pass
        else:
            name = req_body.get('name')

    if name:
        return func.HttpResponse(f"Hello, {name}! Poland GenAI Club is here!")
    else:
        return func.HttpResponse(
             "This HTTP triggered function executed successfully. Pass a name in the query string or in the request body for a personalized response.",
             status_code=200
        )

@app.route(route="hello", methods=["GET"])
def hello(req: func.HttpRequest) -> func.HttpResponse:
    logging.info('Python HTTP trigger function processed a request for /api/hello.')

    return func.HttpResponse("Hello, World!", status_code=200)

@app.route(route="prompt", methods=["POST"])
def process_prompt(req: func.HttpRequest) -> func.HttpResponse:
    logging.info('Python HTTP trigger function processed a request for /api/prompt.')

    # Check if the openai_api_key header is provided
    openai_api_key = req.headers.get("openai_api_key")
    if not openai_api_key:
        return func.HttpResponse("OpenAI API key not provided.", status_code=401)

    # Verify the provided API key (optional step, you may want to implement your own verification logic)
    if openai_api_key != os.getenv("OPENAI_API_KEY"):
        return func.HttpResponse("Invalid OpenAI API key.", status_code=403)

    try:
        req_body = req.get_json()
        prompt = req_body.get('prompt')
    except ValueError:
        return func.HttpResponse("Invalid JSON format in the request body.", status_code=400)

    if prompt:
        response = openai.ChatCompletion.create(
            engine="gpt35_test",
            messages=[
                {"role": "system", "content": "You are an AI assistant that helps people find information."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=800,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None
        )

        return func.HttpResponse(response['choices'][0]['message']['content'], status_code=200)

    return func.HttpResponse("No 'prompt' provided in the request body.", status_code=400)
